In [1]:
import pandas as pd
import xgboost as xgb
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [2]:
sample_submission = pd.read_csv(r"data/sample_submission.csv")
# 提出用ファイルの中身の確認
sample_submission

,ID,取引価格（総額）_log
0,1000000,0
1,1000001,0
2,1000062,0
3,1000114,0
4,1000116,0
...,...,...
23146,47006024,0
23147,47006082,0
23148,47006216,0
23149,47006238,0


In [3]:
#input file name
input_file_name = 'data/data_explanation.xlsx'
#xls book Open (xls, xlsxのどちらでも可能)
input_book = pd.ExcelFile(input_file_name)
#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = input_book.sheet_names
#lenでシートの総数を確認
num_sheet = len(input_sheet_name)
#シートの数とシートの名前のリストの表示
print ("Sheet の数:", num_sheet)
print (input_sheet_name)

Sheet の数: 2
['ファイル説明', 'カラム説明']


In [4]:
#DataFrameとしてsheet1枚のデータ(ファイル説明)を読込み
input_sheet_df = input_book.parse(input_sheet_name[0])
#読み込んだシート名の確認
print("Sheet name:", input_sheet_name[0])
#読み込んだシートの先頭10行を表示
input_sheet_df[['ファイル名', '内容']].head()

Sheet name: ファイル説明


,ファイル名,内容
0,train.zip\n └─*.csv,2020年第４四半期より前に取引された中古マンションの価格データ
1,test.csv,2020年第４四半期、2021年第１四半期に取引された中古マンションの価格データ
2,sample_submission.csv,投稿データフォーマット


In [5]:
#DataFrameとしてsheet1枚のデータ(ファイル説明)を読込み
input_sheet_df = input_book.parse(input_sheet_name[1])
#読み込んだシート名の確認
print("Sheet name:", input_sheet_name[1])

Sheet name: カラム説明


In [8]:
#読み込んだシートの先頭10行を表示
columns_list = input_sheet_df.columns
input_sheet_df[['{}'.format(columns_list[0]), '{}'.format(columns_list[1])]]

,カラム名,内容
0,ID,取引ID
1,種類,不動産の種類
2,地域,周辺地域の状況 ※本データではnull
3,市区町村コード,市区町村コード
4,都道府県名,都道府県名
5,市区町村名,市区町村名
6,地区名,地区名
7,最寄駅：名称,最寄りの鉄道駅の名称
8,最寄駅：距離（分）,当該地から最寄りの鉄道駅（地下駅の場合には地表への出入口）までの時間距離（分）
9,間取り,専有部分の間取りを表示しています。なお、間取りが把握できないものは、空欄になっています。\n...


In [7]:
test_df = pd.read_csv('data/test.csv')
print(test_df.head())
print('columns:{}'.format(test_df.columns))

        ID        種類  地域  市区町村コード 都道府県名   市区町村名  地区名 最寄駅：名称 最寄駅：距離（分）   間取り  \
0  1000000  中古マンション等 NaN     1101   北海道  札幌市中央区  旭ケ丘   円山公園        25  ４ＬＤＫ   
1  1000001  中古マンション等 NaN     1101   北海道  札幌市中央区  旭ケ丘   円山公園        20  ４ＬＤＫ   
2  1000062  中古マンション等 NaN     1101   北海道  札幌市中央区  大通西  西１１丁目         3  ２ＬＤＫ   
3  1000114  中古マンション等 NaN     1101   北海道  札幌市中央区  大通西  西１８丁目         2  １ＬＤＫ   
4  1000116  中古マンション等 NaN     1101   北海道  札幌市中央区  大通西  西１８丁目         4  ２ＬＤＫ   

   ... 今後の利用目的  前面道路：方位  前面道路：種類  前面道路：幅員（ｍ）         都市計画 建ぺい率（％） 容積率（％）  \
0  ...      住宅      NaN      NaN         NaN  第１種低層住居専用地域    40.0   80.0   
1  ...      住宅      NaN      NaN         NaN       近隣商業地域    80.0  200.0   
2  ...      住宅      NaN      NaN         NaN         商業地域    80.0  600.0   
3  ...      住宅      NaN      NaN         NaN         商業地域    80.0  400.0   
4  ...      住宅      NaN      NaN         NaN       近隣商業地域    80.0  300.0   

         取引時点   改装  取引の事情等  
0  2021年第１四半期  未改装     NaN  
1  2021年第１

In [8]:
import glob
train_files = glob.glob('data/train/*.csv')
print(train_files)

['data/train/40.csv', 'data/train/41.csv', 'data/train/43.csv', 'data/train/42.csv', 'data/train/46.csv', 'data/train/47.csv', 'data/train/45.csv', 'data/train/44.csv', 'data/train/37.csv', 'data/train/23.csv', 'data/train/22.csv', 'data/train/36.csv', 'data/train/08.csv', 'data/train/20.csv', 'data/train/34.csv', 'data/train/35.csv', 'data/train/21.csv', 'data/train/09.csv', 'data/train/25.csv', 'data/train/31.csv', 'data/train/19.csv', 'data/train/18.csv', 'data/train/30.csv', 'data/train/24.csv', 'data/train/32.csv', 'data/train/26.csv', 'data/train/27.csv', 'data/train/33.csv', 'data/train/16.csv', 'data/train/02.csv', 'data/train/03.csv', 'data/train/17.csv', 'data/train/29.csv', 'data/train/01.csv', 'data/train/15.csv', 'data/train/14.csv', 'data/train/28.csv', 'data/train/04.csv', 'data/train/10.csv', 'data/train/38.csv', 'data/train/39.csv', 'data/train/11.csv', 'data/train/05.csv', 'data/train/13.csv', 'data/train/07.csv', 'data/train/06.csv', 'data/train/12.csv']


In [9]:
train_df_list = []
for file in train_files:
    df = pd.read_csv(file)
    train_df_list.append(df)
train_df = pd.concat(train_df_list)

/Users/masato/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
print('train_df:\n{}'.format(train_df.columns))
print('test_df:\n{}'.format(test_df.columns))
print('sample_df:\n{}'.format(sample_submission.columns))

train_df:
Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')
test_df:
Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等'],
      dtype='object')
sample_df:
Index(['ID', '取引価格（総額）_log'], dtype='object')


In [11]:
train_df['test'] = 0
test_df['test'] = 1
df = pd.concat([train_df, test_df])

In [12]:

def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[-1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[-1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[-1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    cols = ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
    ce_df = ce.OrdinalEncoder(cols=cols, handle_unknown='impute')
    df = ce_df.fit_transform(df)
    
    return df
    
df = data_pre(df)
train_df = df[df['test'] == 0]
test_df = df[df['test'] == 1]


In [13]:
train_df = train_df.drop("test", axis = 1)
test_df = test_df.drop(["test", "取引価格（総額）_log"], axis = 1)
train_df.head()

,ID,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,40012297,40131,1,1,1,6.0,1,60.0,25.0,1,1,1,1,NaN,NaN,2017.25,1,1,6.919078
1,40067189,40133,1,2,2,4.0,2,25.0,8.0,1,1,2,2,80.0,400.0,2014.75,1,1,7.230449
2,40019754,40133,1,3,3,3.0,3,65.0,5.0,1,2,1,2,80.0,400.0,2019.75,1,1,7.643453
3,40097300,40132,1,4,4,16.0,4,15.0,26.0,1,1,2,3,80.0,300.0,2013.75,1,1,6.544068
4,40017646,40133,1,5,5,8.0,2,60.0,17.0,1,1,1,2,80.0,400.0,2016.25,1,1,7.462398


In [37]:
def return_predict_by_XGBoost(train_df, test_df, sample_submission):
    df_train, df_val =train_test_split(train_df, test_size=0.2)

    col = "取引価格（総額）_log"
    train_y = df_train[col]
    train_x = df_train.drop(col, axis=1)

    val_y = df_val[col]
    val_x = df_val.drop(col, axis=1)

    train_data = xgb.DMatrix(train_x, label=train_y)
    eval_data = xgb.DMatrix(val_x, label=val_y)

    xgb_params = {
        "objective": "reg:squarederror",
        'eval_metric': "mae"
        }
    evals = [(train_data, 'train'), (eval_data, 'eval')]

    gbm = xgb.train(
        xgb_params,
        train_data,
        num_boost_round=100,
        early_stopping_rounds=10,
        evals=evals,
        )
    
    predict_X = xgb.DMatrix(test_df)
    y_pred = gbm.predict(predict_X)
    columns = sample_submission.columns
    for i in range(len(y_pred)):
        sample_submission[columns[1]][i] = y_pred[i]
        
    return sample_submission

In [39]:
sample_submission = return_predict_by_XGBoost(train_df, test_df, sample_submission)
sample_submission.to_csv('submission2.csv', index=False)

[0]	train-mae:4.70596	eval-mae:4.70532
[1]	train-mae:3.29434	eval-mae:3.29379
[2]	train-mae:2.30623	eval-mae:2.30589
[3]	train-mae:1.61463	eval-mae:1.61427
[4]	train-mae:1.13094	eval-mae:1.13062
[5]	train-mae:0.79403	eval-mae:0.79377
[6]	train-mae:0.56000	eval-mae:0.55979
[7]	train-mae:0.39865	eval-mae:0.39850
[8]	train-mae:0.28987	eval-mae:0.28979
[9]	train-mae:0.21936	eval-mae:0.21931
[10]	train-mae:0.17519	eval-mae:0.17521
[11]	train-mae:0.14844	eval-mae:0.14847
[12]	train-mae:0.13329	eval-mae:0.13336
[13]	train-mae:0.12458	eval-mae:0.12467
[14]	train-mae:0.11990	eval-mae:0.12001
[15]	train-mae:0.11675	eval-mae:0.11685
[16]	train-mae:0.11449	eval-mae:0.11464
[17]	train-mae:0.11231	eval-mae:0.11248
[18]	train-mae:0.11103	eval-mae:0.11125
[19]	train-mae:0.11010	eval-mae:0.11035
[20]	train-mae:0.10935	eval-mae:0.10964
[21]	train-mae:0.10745	eval-mae:0.10780
[22]	train-mae:0.10680	eval-mae:0.10718
[23]	train-mae:0.10616	eval-mae:0.10656
[24]	train-mae:0.10561	eval-mae:0.10601
[25]	train

<ipython-input-37-64a2fcb0cbe5>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_submission[columns[1]][i] = y_pred[i]
